In [26]:
import os
import json
import time
import requests
import urllib.parse
from IPython.display import Image, display
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


# Azure Maps API
subscriptionKey = "mdvhwX6Era11JlFA5Mtx9Lar7xnzUxQc_6fuvtQ8p_w"

# load the shapefile of the city polygon
#city = gpd.read_file("C:/Users/Magdalenka Kolacik/Jankyna zaloha/2023/Hack Kosice/get_geojson.geojson")

#url = "https://github.com/caliline2/Hack_Kosice/blob/main/get_geojson.geojson"
#https://github.com/caliline2/Hack_Kosice/blob/main/get_geojson.geojson
#city = gpd.read_file(url)

#print(gdf.head())

# Load category matrix
#matrix = pd.read_excel("C:/Users/Magdalenka Kolacik/Jankyna zaloha/2023/Hack Kosice/DataTypesAzureMap.xlsx")

url2 = "https://raw.githubusercontent.com/caliline2/Hack_Kosice/main/DataTypesAzureMap.csv"

matrix = pd.read_csv(url2)

# User input address
addresstoAnalyze = input("Enter an address to analyze: ") #Analyzing the Catchment Area for the Starbucks Store (400, Pine Street, Seattle) in Westlake Center, Seattle. 
locationDetails = requests.get("https://atlas.microsoft.com/search/address/json?subscription-key={}&api-version=1.0&query={}".format(subscriptionKey,addresstoAnalyze)).json()
latlong=locationDetails["results"][0]["position"]
currentLocation=[0,0]
currentLocation[0]=latlong["lat"]
currentLocation[1]=latlong["lon"]

# return lat long of the point (to be printed in front end)

print("Lat Long for",addresstoAnalyze,"is",latlong)
walkingTime = 300 # this is 600secs. you can experiment with different values
cTraffic = "true" #you can test how far you can go in current traffic
tMode = "pedestrian" # you can change this to car, bus, pedestrian, bicycle and more. see http://bit.ly/2nGErtX for details 


# Calculate polygon x minutes distance
routeRangeResponse = requests.get("https://atlas.microsoft.com/route/range/json?subscription-key={}&api-version=1.0&query={}&timeBudgetInSec={}&traffic={}&travelmode={}".format(subscriptionKey,str(currentLocation[0])+","+str(currentLocation[1]),walkingTime,cTraffic,tMode)).json()
routeRangeResponseF = json.dumps(routeRangeResponse, indent=4)
print(routeRangeResponseF)


polyBounds = routeRangeResponse["reachableRange"]["boundary"]


for i in range(len(polyBounds)):
    coordList = list(polyBounds[i].values())
    coordList[0], coordList[1] = coordList[1], coordList[0]
    polyBounds[i] = coordList

polyBounds.pop()
polyBounds.append(polyBounds[0])

boundsData = {
               "geometry": {
                 "type": "Polygon",
                 "coordinates": 
                   [
                      polyBounds
                   ]
                }
             }

boundsDataF = json.dumps(boundsData, indent=4)
print(boundsDataF)


poiNames = ["ACCESS_GATEWAY",
"ADMINISTRATIVE_DIVISION",
"ADVENTURE_SPORTS_VENUE",
"AGRICULTURE",
"AIRPORT",
"AMUSEMENT_PARK",
"AUTOMOTIVE_DEALER",
"BANK",
"BEACH",
"BUILDING_POINT",
"BUSINESS_PARK",
"CAFE_PUB",
"CAMPING_GROUND",
"CAR_WASH",
"CASH_DISPENSER",
"CASINO",
"CINEMA",
"CITY_CENTER",
"CLUB_ASSOCIATION",
"COLLEGE_UNIVERSITY",
"COMMERCIAL_BUILDING",
"COMMUNITY_CENTER",
"COMPANY",
"COURTHOUSE",
"CULTURAL_CENTER",
"DENTIST",
"DEPARTMENT_STORE",
"DOCTOR",
"ELECTRIC_VEHICLE_STATION",
"EMBASSY",
"EMERGENCY_MEDICAL_SERVICE",
"ENTERTAINMENT",
"EXCHANGE",
"EXHIBITION_CONVENTION_CENTER",
"FERRY_TERMINAL",
"FIRE_STATION_BRIGADE",
"FRONTIER_CROSSING",
"FUEL_FACILITIES",
"GEOGRAPHIC_FEATURE",
"GOLF_COURSE",
"GOVERNMENT_OFFICE",
"HEALTH_CARE_SERVICE",
"HELIPAD_HELICOPTER_LANDING",
"HOLIDAY_RENTAL",
"HOSPITAL_POLYCLINIC",
"HOTEL_MOTEL",
"ICE_SKATING_RINK",
"IMPORTANT_TOURIST_ATTRACTION",
"INDUSTRIAL_BUILDING",
"LEISURE_CENTER",
"LIBRARY",
"MANUFACTURING_FACILITY",
"MARINA",
"MARKET",
"MEDIA_FACILITY",
"MILITARY_INSTALLATION",
"MOTORING_ORGANIZATION_OFFICE",
"MOUNTAIN_PASS",
"MUSEUM",
"NATIVE_RESERVATION",
"NIGHTLIFE",
"NON_GOVERNMENTAL_ORGANIZATION",
"OPEN_PARKING_AREA",
"OTHER",
"PARKING_GARAGE",
"PARK_RECREATION_AREA",
"PETROL_STATION",
"PHARMACY",
"PLACE_OF_WORSHIP",
"POLICE_STATION",
"PORT_WAREHOUSE_FACILITY",
"POST_OFFICE",
"PRIMARY_RESOURCE_UTILITY",
"PRISON_CORRECTIONAL_FACILITY",
"PUBLIC_AMENITY",
"PUBLIC_TRANSPORT_STOP",
"RAILWAY_STATION",
"RENT_A_CAR_FACILITY",
"RENT_A_CAR_PARKING",
"REPAIR_FACILITY",
"RESEARCH_FACILITY",
"RESIDENTIAL_ACCOMMODATION",
"RESTAURANT",
"RESTAURANT_AREA",
"REST_AREA",
"SCENIC_PANORAMIC_VIEW",
"SCHOOL",
"SHOP",
"SHOPPING_CENTER",
"SPORTS_CENTER",
"STADIUM",
"SWIMMING_POOL",
"TENNIS_COURT",
"THEATER",
"TOURIST_INFORMATION_OFFICE",
"TRAFFIC_LIGHT",
"TRAFFIC_SERVICE_CENTER",
"TRAFFIC_SIGN",
"TRAIL_SYSTEM",
"TRANSPORT_AUTHORITY",
"VEHICLE_REGISTRATION",
"TRUCK_STOP",
"VETERINARIAN",
"WATER_SPORT",
"WEIGH_STATION",
"WELFARE_ORGANIZATION",
"WINERY",
"ZOOS_ARBORETA_BOTANICAL_GARDEN",]



# Calculating count = counting
# Observations = to display on the map

n = 0
counting = pd.DataFrame(columns=['Category', 'Count'])
observations = pd.DataFrame(columns=['Category','Name','Latitude','Longitude'])

for n in range(106):
    if n >= len(poiNames):
        break
    poiName = poiNames[n]
    searchPolyResponse = requests.post(url = "https://atlas.microsoft.com/search/geometry/json?subscription-key={}&api-version=1.0&query={}&idxSet=POI&limit=50".format(subscriptionKey, poiName), json = boundsData).json()
    numRes = searchPolyResponse["summary"]["numResults"]
    print("Total Number of "+ poiName +" nearby: ", numRes)
    counting= counting.append({'Category': poiName, 'Count': numRes}, ignore_index=True)    
    print("Here is a list of the Search Results: ")
    for loc in range(len(searchPolyResponse["results"])):
                print(loc+1,". ",searchPolyResponse["results"][loc]["poi"]["name"])
    for loc in range(len(searchPolyResponse["results"])):
        print(loc+1,". ",searchPolyResponse["results"][loc]["poi"]["name"])
        latitude = searchPolyResponse["results"][loc]["position"]["lat"]
        longitude = searchPolyResponse["results"][loc]["position"]["lon"]
        print("Latitude: ", latitude)
        print("Longitude: ", longitude)
        observations = counting.append({'Category': poiName, 'Name': (loc+1,". ",searchPolyResponse["results"][loc]["poi"]["name"]),'Latitude':latitude,'Longitude':longitude}, ignore_index=True)
    
    
        
        
print(counting)
    #observations = observations.dropna(subset=['Name'], inplace=True)
    #print(observations)
    
# Merging matrix with counting per polygon
ranking = pd.merge(counting, matrix, on='Category')
    
ranking['Industrial_Rank'] = ranking['Scoring_industrial'] * ranking['Count']
ranking['F&B'] = ranking['Scoring_F&B'] * ranking['Count']
ranking['Scoring_amenities'] = ranking['Scoring_amenities'] * ranking['Count']

# Normalizing to 0-100
Industrial_Ranking = ranking['Industrial_Rank'].sum()
Industrial_Ranking = min(Industrial_Ranking/500, 1) * 100   # normalize score to 0-100 range
Industrial_Ranking = min(Industrial_Ranking, 100)   # cap the score at 100

FoodandBeverage_Ranking = ranking['F&B'].sum()
FoodandBeverage_Ranking = min(FoodandBeverage_Ranking/500, 1) * 100   # normalize score to 0-100 range
FoodandBeverage_Ranking = min(FoodandBeverage_Ranking, 100)   # cap the score at 100

Public_Amenities_Ranking = ranking['Scoring_amenities'].sum()
Public_Amenities_Ranking = min(Public_Amenities_Ranking/500, 1) * 100   # normalize score to 0-100 range
Public_Amenities_Ranking = min(Public_Amenities_Ranking, 100)   # cap the score at 100

print("Industrial Ranking is ", Industrial_Ranking)
print("F&B Ranking is ", FoodandBeverage_Ranking)
print("Public Amenities Ranking is ", Public_Amenities_Ranking)




Enter an address to analyze: Hlavná 63 Staré Mesto, 040 01 Staré Mesto
Lat Long for Hlavná 63 Staré Mesto, 040 01 Staré Mesto is {'lat': 48.72187, 'lon': 21.25778}
{
    "formatVersion": "0.0.1",
    "reachableRange": {
        "center": {
            "latitude": 48.72184,
            "longitude": 21.25766
        },
        "boundary": [
            {
                "latitude": 48.72452,
                "longitude": 21.25764
            },
            {
                "latitude": 48.72413,
                "longitude": 21.25736
            },
            {
                "latitude": 48.72403,
                "longitude": 21.25706
            },
            {
                "latitude": 48.72389,
                "longitude": 21.25663
            },
            {
                "latitude": 48.72529,
                "longitude": 21.25561
            },
            {
                "latitude": 48.72412,
                "longitude": 21.2559
            },
            {
                

Total Number of ACCESS_GATEWAY nearby:  0
Here is a list of the Search Results: 
Total Number of ADMINISTRATIVE_DIVISION nearby:  0
Here is a list of the Search Results: 
Total Number of ADVENTURE_SPORTS_VENUE nearby:  2
Here is a list of the Search Results: 
1 .  Mc Group 8 SRO
2 .  Biliard Club Katka
1 .  Mc Group 8 SRO
Latitude:  48.72085
Longitude:  21.25897
2 .  Biliard Club Katka
Latitude:  48.7233
Longitude:  21.25759
Total Number of AGRICULTURE nearby:  0
Here is a list of the Search Results: 
Total Number of AIRPORT nearby:  0
Here is a list of the Search Results: 
Total Number of AMUSEMENT_PARK nearby:  0
Here is a list of the Search Results: 
Total Number of AUTOMOTIVE_DEALER nearby:  0
Here is a list of the Search Results: 
Total Number of BANK nearby:  7
Here is a list of the Search Results: 
1 .  VÚB Hlavná 2
2 .  ČSOB Hlavná
3 .  ČSOB Mlynská
4 .  S Slovensko
5 .  UniCredit Bank Slovakia
6 .  Prvá Stavebná Sporiteľňa
7 .  UniCredit Bank Mlynská
1 .  VÚB Hlavná 2
Latitude

Total Number of CAMPING_GROUND nearby:  1
Here is a list of the Search Results: 
1 .  Hostinec Stará Krčma
1 .  Hostinec Stará Krčma
Latitude:  48.71933
Longitude:  21.25853
Total Number of CAR_WASH nearby:  7
Here is a list of the Search Results: 
1 .  ČSOB Hlavná
2 .  Tatra banka Hlavná
3 .  ČSOB Hlavná 85
4 .  VÚB Hlavná 41
5 .  ČSOB Mlynská
6 .  UniCredit Bank Mlynská
7 .  Slovenská sporiteľňa Mlynská
1 .  ČSOB Hlavná
Latitude:  48.7195
Longitude:  21.25914
2 .  Tatra banka Hlavná
Latitude:  48.72158
Longitude:  21.258
3 .  ČSOB Hlavná 85
Latitude:  48.72369
Longitude:  21.2566
4 .  VÚB Hlavná 41
Latitude:  48.72074
Longitude:  21.2585
5 .  ČSOB Mlynská
Latitude:  48.72121
Longitude:  21.26038
6 .  UniCredit Bank Mlynská
Latitude:  48.72092
Longitude:  21.25925
7 .  Slovenská sporiteľňa Mlynská
Latitude:  48.72076
Longitude:  21.25938
Total Number of CASH_DISPENSER nearby:  7
Here is a list of the Search Results: 
1 .  ČSOB Hlavná
2 .  Tatra banka Hlavná
3 .  ČSOB Hlavná 85
4 .  VÚ

37 .  Eu - Tax, AS
Latitude:  48.72167
Longitude:  21.25675
38 .  Aukčná A Konzultačná, Spol. S RO
Latitude:  48.72263
Longitude:  21.25873
39 .  Profi Hockey SRO
Latitude:  48.7228
Longitude:  21.25897
40 .  Real Consulting House SRO
Latitude:  48.72414
Longitude:  21.2577
41 .  Slovenská Dražobná, Spol. S RO
Latitude:  48.71897
Longitude:  21.25836
42 .  Výťahy Slovensko SRO
Latitude:  48.72372
Longitude:  21.25818
43 .  Kirchhoff, SRO
Latitude:  48.72255
Longitude:  21.25984
44 .  Service Medical Slovakia SRO
Latitude:  48.71872
Longitude:  21.25859
45 .  Honor, SRO
Latitude:  48.72062
Longitude:  21.25881
46 .  Perun & B, Verejná Obchodná Spoločnosť, Košice
Latitude:  48.72249
Longitude:  21.25471
47 .  Cedre SRO
Latitude:  48.72085
Longitude:  21.25898
48 .  Strandker KFT Organizačná Zložka Košice
Latitude:  48.71872
Longitude:  21.25859
49 .  La Puerta, S. R. O.
Latitude:  48.72268
Longitude:  21.25903
50 .  Polservice S. R. O.
Latitude:  48.72045
Longitude:  21.26074
Total Numbe

Total Number of ELECTRIC_VEHICLE_STATION nearby:  0
Here is a list of the Search Results: 
Total Number of EMBASSY nearby:  1
Here is a list of the Search Results: 
1 .  Generálny Konzulát Maďarskej Republiky V Košiciach
1 .  Generálny Konzulát Maďarskej Republiky V Košiciach
Latitude:  48.72423
Longitude:  21.25632
Total Number of EMERGENCY_MEDICAL_SERVICE nearby:  50
Here is a list of the Search Results: 
1 .  MUDr. Lucia Bohušová - Stomatológia, SRO
2 .  Bernardclub
3 .  Jt Group SRO
4 .  Reward, SRO
5 .  Sírius Spoločnosť S Ručením Obmedzeným Košice
6 .  Denal, SRO
7 .  Orter Slovakia, SRO
8 .  Fam, SRO
9 .  La Medusa SRO
10 .  Ceeb Slovakia, SRO
11 .  Chimera Group SRO
12 .  Art-Optika - Gabriela Artimová
13 .  OMO Plus, SRO
14 .  Plastic Hobby Kits Spol.S RO
15 .  Topas.Sk, SRO
16 .  Domino Club - Družstvo Košice
17 .  Break Limits, S. R. O.
18 .  Najavo, SRO
19 .  Herberia, SRO
20 .  Trisso Invest - Družstvo
21 .  Authentic Kosice
22 .  Tatiana Dent, SRO
23 .  Obelisk, SRO
24 . 

...

Total Number of PRISON_CORRECTIONAL_FACILITY nearby:  2
Here is a list of the Search Results: 
1 .  Miklus Prison
2 .  Mikluš Prison
1 .  Miklus Prison
Latitude:  48.72282
Longitude:  21.25962
2 .  Mikluš Prison
Latitude:  48.7229
Longitude:  21.25957
Total Number of PUBLIC_AMENITY nearby:  0
Here is a list of the Search Results: 
Total Number of PUBLIC_TRANSPORT_STOP nearby:  2
Here is a list of the Search Results: 
1 .  Vodná Záchranná Služba SRO
2 .  Transport Solutions SRO
1 .  Vodná Záchranná Služba SRO
Latitude:  48.72317
Longitude:  21.25687
2 .  Transport Solutions SRO
Latitude:  48.72238
Longitude:  21.25994
Total Number of RAILWAY_STATION nearby:  0
Here is a list of the Search Results: 
Total Number of RENT_A_CAR_FACILITY nearby:  33
Here is a list of the Search Results: 
1 .  Bernardclub
2 .  Univerzal Sls, Spoločnosť S Ručením Obmedzeným Košice
3 .  Reward, SRO
4 .  Smaragd, SRO
5 .  Mc Group 8 SRO
6 .  Stafin SRO
7 .  Čo Potrebuješ ?!, Spol. S RO
8 .  Najavo, SRO
9 .  blu

Total Number of REPAIR_FACILITY nearby:  0
Here is a list of the Search Results: 
Total Number of RESEARCH_FACILITY nearby:  0
Here is a list of the Search Results: 
Total Number of RESIDENTIAL_ACCOMMODATION nearby:  0
Here is a list of the Search Results: 
Total Number of RESTAURANT nearby:  50
Here is a list of the Search Results: 
1 .  Caravella
2 .  Burrito
3 .  Take It
4 .  Alchymista
5 .  Barrique-Pivoteka & Vinoteka
6 .  Vega Heaven
7 .  Luxor Pivaren
8 .  Muse Košice
9 .  Karczma Mlyn
10 .  U Slona
11 .  Himalaya Restro Bar & Shisha Lounge
12 .  Central Pub & Restaurant
13 .  Pablo restaurant & garden
14 .  Haluskaren
15 .  Zlatý bažant
16 .  Beer House
17 .  Café de Paris
18 .  Pekarnicka Na Korze
19 .  Cafe Racine
20 .  Vitézov Dvor
21 .  Cervena Kalina
22 .  Jedáleň Fantázia
23 .  Massimo Street Food
24 .  Villa Cassa Wine Bar And Shop
25 .  U starého otca
26 .  Babetta Kgp
27 .  Camelot-Medieval Restaurant
28 .  The Porterhouse Košice
29 .  Gallery club
30 .  Marrakesh
31 .

Total Number of REST_AREA nearby:  4
Here is a list of the Search Results: 
1 .  Staré Mesto
2 .  Bristol
3 .  Vodná
4 .  Hostinec Stará Krčma
1 .  Staré Mesto
Latitude:  48.72316
Longitude:  21.25791
2 .  Bristol
Latitude:  48.72046
Longitude:  21.25971
3 .  Vodná
Latitude:  48.72427
Longitude:  21.25855
4 .  Hostinec Stará Krčma
Latitude:  48.71933
Longitude:  21.25853
Total Number of SCENIC_PANORAMIC_VIEW nearby:  0
Here is a list of the Search Results: 
Total Number of SCHOOL nearby:  10
Here is a list of the Search Results: 
1 .  Jazyková škola
2 .  ZŠ s VJM-Alapiskola
3 .  SPŠ dopravná
4 .  MŠ s VJM-Óvoda
5 .  Skrivanek Slovensko, SRO
6 .  Premonštrátske gymnáziu
7 .  SZŠ sv. Alžbety
8 .  Materská škola Hrnčiarska
9 .  Základná Umelecká Škola Márie Hemerkovej
10 .  Základná umelecká škola Kováčska
1 .  Jazyková škola
Latitude:  48.71931
Longitude:  21.25917
2 .  ZŠ s VJM-Alapiskola
Latitude:  48.72117
Longitude:  21.25697
3 .  SPŠ dopravná
Latitude:  48.71931
Longitude:  21.25917

Total Number of SPORTS_CENTER nearby:  7
Here is a list of the Search Results: 
1 .  Mc Group 8 SRO
2 .  Crystal - Štúdio Zdravia A Vitality
3 .  Kupele Sklene Teplice
4 .  Himaláj Centrum - Chránená Dielňa
5 .  Biliard Club Katka
6 .  Privat in Kosice centre
7 .  Centrum Apartments
1 .  Mc Group 8 SRO
Latitude:  48.72085
Longitude:  21.25897
2 .  Crystal - Štúdio Zdravia A Vitality
Latitude:  48.72139
Longitude:  21.26112
3 .  Kupele Sklene Teplice
Latitude:  48.7222
Longitude:  21.25758
4 .  Himaláj Centrum - Chránená Dielňa
Latitude:  48.72332
Longitude:  21.25868
5 .  Biliard Club Katka
Latitude:  48.7233
Longitude:  21.25759
6 .  Privat in Kosice centre
Latitude:  48.72168
Longitude:  21.25537
7 .  Centrum Apartments
Latitude:  48.72291
Longitude:  21.25713
Total Number of STADIUM nearby:  0
Here is a list of the Search Results: 
Total Number of SWIMMING_POOL nearby:  0
Here is a list of the Search Results: 
Total Number of TENNIS_COURT nearby:  1
Here is a list of the Search Resu

Total Number of TRAFFIC_SIGN nearby:  0
Here is a list of the Search Results: 
Total Number of TRAIL_SYSTEM nearby:  1
Here is a list of the Search Results: 
1 .  Trade Trans Rail Spoločnosť S Ručením Obmedzeným Košice
1 .  Trade Trans Rail Spoločnosť S Ručením Obmedzeným Košice
Latitude:  48.72414
Longitude:  21.2577
Total Number of TRANSPORT_AUTHORITY nearby:  2
Here is a list of the Search Results: 
1 .  Vodná Záchranná Služba SRO
2 .  Transport Solutions SRO
1 .  Vodná Záchranná Služba SRO
Latitude:  48.72317
Longitude:  21.25687
2 .  Transport Solutions SRO
Latitude:  48.72238
Longitude:  21.25994
Total Number of VEHICLE_REGISTRATION nearby:  0
Here is a list of the Search Results: 
Total Number of TRUCK_STOP nearby:  50
Here is a list of the Search Results: 
1 .  Spoločenstvo Vlastníkov Bytov Fregata - Košice
2 .  Gastrokol SRO
3 .  Spoločenstvo Vlastníkov Bytov A Nebytových Priestorov Hegedűs Dom
4 .  Univerzal Sls, Spoločnosť S Ručením Obmedzeným Košice
5 .  Eurodomos, AS
6 .  

Enter an address to analyze: kopanice 21, bratislava, 82104


Lat Long for kopanice 21, bratislava, 82104 is {'lat': 48.16834, 'lon': 17.15988}
